In [1]:
import numpy as np
import math
import matplotlib as plt
import pickle
import statistics
import os
import glob
import pandas as pd

In [91]:
data = pd.read_pickle("normalized_features_final.pkl")
#data

data2 = pd.read_pickle("features_final_no_norm.pkl")
#data2

In [44]:
#computes euclidean distance between 2 vectors
def euclidean_distance(vector1, vector2):
    euclidean = np.linalg.norm(np.asarray(vector1[2:9]) - np.asarray(vector2[2:9]))
    return euclidean

#computes cosine distance between 2 vectors
def cosine_distance(vector1, vector2):
    one_arr = np.asarray(vector1)
    two_arr = np.asarray(vector2)

    #scalar_weight = 1 / 12
    #weights = np.append(np.full(shape=7, fill_value=(scalar_weight)), np.full(shape=500, fill_value=(scalar_weight / 100)))
    #one_arr = one_arr * weights
    #two_arr = two_arr * weights
    cosine = 1 - ((one_arr @ two_arr) / (np.linalg.norm(one_arr) * np.linalg.norm(two_arr)))
    return cosine

In [88]:
def scale_dict(my_dict, max_val):
    for i in my_dict:
        my_dict[i] = float(my_dict[i]/max_val)
    return my_dict 

def combine_dicts(dict1, dict2):
    new_dict = {}
    for i in dict1:
        mean_distance = (dict1[i] + dict2[i])/2
        new_dict[i] = mean_distance 
    return new_dict

#returns top k matches
def match(vector1, k):
    distance_dictionary = {}
    distance_dictionary_2 = {}
    for i in range(len(data)):
        vector2 = data.iloc[i]
        match_path = vector2[1]
        distance = cosine_distance(vector1[2:9], vector2[2:9])
        distance_dictionary[match_path] = distance

        distance_2 = euclidean_distance(vector1[2:9], vector2[2:9])
        distance_dictionary_2[match_path] = distance_2

    #combining the two distance dicts
    max_value = max(distance_dictionary_2, key=distance_dictionary_2.get)
    maximum = distance_dictionary_2[max_value]
    distance_dictionary_2 = scale_dict(distance_dictionary_2, maximum)
    new_dict = combine_dicts(distance_dictionary, distance_dictionary_2)

    # Sort the dictionary by value using a lambda function to extract the values
    result = dict(sorted(distance_dictionary.items(), key = lambda x: x[1], reverse = False)[:k])
    return result


In [95]:
vector1 = data.iloc[2]
print("input object path:", vector1[1])
result = match(vector1, 10)
print("top k closest matches in database (paths+cosine distance):")
for i in list(result.items()):
    print(i)

input object path: ./features/AircraftBuoyant\m1339
top k closest matches in database (paths+cosine distance):
('./features/AircraftBuoyant\\m1339', -2.220446049250313e-16)
('./features/Insect\\D00109', 0.011747381260536871)
('./features/Hat\\m1640', 0.042311048282604236)
('./features/AircraftBuoyant\\m1345', 0.05316887771273271)
('./features/RoundTable\\D00649', 0.05655049073037477)
('./features/Starship\\m1383', 0.057071643244073655)
('./features/Hat\\m1629', 0.07086972725311957)
('./features/Tree\\D00096', 0.07403415485066367)
('./features/HumanHead\\m367', 0.07491248074400703)
('./features/Insect\\D00102', 0.07592997278520575)


In [29]:
def flatten(vec):
    output = [vec[2], vec[3], vec[4], vec[5], vec[6], vec[7], vec[8]]
    output.extend(vec[9])
    output.extend(vec[10])
    output.extend(vec[11])
    output.extend(vec[12])
    output.extend(vec[13])
    return output

100
507


In [47]:
#returns top k matches
def match_test(vector1, k):
    vector1 = flatten(vector1)
    distance_dictionary = {}
    for i in range(len(data)):
        vector2 = data.iloc[i]
        match_path = vector2[1]
        vector2 = flatten(vector2)
        
        distance = euclidean_distance(vector1, vector2)
        distance_dictionary[match_path] = distance
        
    # Sort the dictionary by value using a lambda function to extract the values
    result = dict(sorted(distance_dictionary.items(), key = lambda x: x[1], reverse = False)[:k])
    return result

vector1 = data.iloc[1000]
match_test(vector1, 10)

{'./features/House\\m448': 0.0,
 './features/Musical_Instrument\\m644': 0.12238787259691584,
 './features/FloorLamp\\D00093': 0.2932603932248084,
 './features/Mug\\m508': 0.36045628313698785,
 './features/Vase\\m544': 0.3853557551368405,
 './features/NonWheelChair\\m799': 0.45013940731346797,
 './features/PlantWildNonTree\\m1052': 0.4614051862866583,
 './features/PlantIndoors\\m1027': 0.46755340133572737,
 './features/NonWheelChair\\m796': 0.4880073933374033,
 './features/BuildingNonResidential\\m415': 0.496067330305771}